In [1]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta
%matplotlib inline 

##### DATA (the anonymised dataframe) 

In [2]:
#import main dataframe
data = pd.read_csv('locations.csv', low_memory=False)

In [3]:
#rename City column and drop empty columns
data.rename( columns={'Unnamed: 5':'City'}, inplace=True )
data = data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1)

In [4]:
#drop VisitorID and ContentInfo
data = data.drop(columns=['VisitorID', 'ContentInfo'])

In [5]:
#just getting rid of the space after the name
data = data.replace('STH Ayshire St Leonards ', 'STH Ayshire St Leonards')

In [6]:
#change time to datetime
data['Time'] = pd.to_datetime(data['Time'], format = "%m/%d/%y %H:%M", utc=True)

In [7]:
#change ExternalIDs to integers
data.ExternalID = data.ExternalID.fillna(0).astype(int)

In [8]:
#drop rows that are identical
data = data.drop_duplicates(keep='first')

In [9]:
#new dataframe with no null ExternalID vaues
dataNoNullUsers = data[data.ExternalID != 0]

##### USERS (the user dataframe) 

In [10]:
#import users dataframe
users = pd.read_csv('PrimaryConditions age sex by customer reference.csv', low_memory=False)

In [11]:
#drop empty column and CleverCogsUserId
users = users.drop(users.columns[users.columns.str.contains('unnamed',case = False)],axis = 1)
users = users.drop(columns=['CleverCogsUserId'])

In [12]:
#delete empty row
users = users.dropna(axis=0, how='all', inplace=False)

In [13]:
#change ExternalID into integers
users['ExternalID'] = users['ExternalID'].apply(np.int64)
users.shape
users.head()

,ExternalID,BirthDate,Gender,Condition
0,4703,7/4/65,F,Spina Bifida
1,4704,6/21/40,F,COPD
2,4706,2/25/31,F,Dementia
3,4707,9/18/82,F,Brain Injury
4,4708,12/20/31,F,NaN


In [14]:
#drop duplicates
users = users.drop_duplicates(subset = ['ExternalID'], keep='last')
users.shape

(680, 4)

In [15]:
#change BirthDate to date time
users['BirthDate'] = pd.to_datetime(users['BirthDate'], format = "%m/%d/%y", dayfirst=False, utc=True)

# users['BirthDate'] = pd.to_datetime(users['BirthDate'], errors='coerce', dayfirst=False, yearfirst=False, 
#                    format="%d/%m/%Y", utc=False, infer_datetime_format=False, origin='unix')
# users = users.set_index(pd.DatetimeIndex(users['BirthDate']))

# Remove erroneous entries
# threshold = pd.to_datetime("2010-01-01", format="%Y-%m-%d", utc=False, origin='unix')
# users = users[users['BirthDate'] < threshold]

users.head()

,ExternalID,BirthDate,Gender,Condition
0,4703,2065-07-04 00:00:00+00:00,F,Spina Bifida
1,4704,2040-06-21 00:00:00+00:00,F,COPD
2,4706,2031-02-25 00:00:00+00:00,F,Dementia
3,4707,1982-09-18 00:00:00+00:00,F,Brain Injury
4,4708,2031-12-20 00:00:00+00:00,F,NaN


In [16]:
now = pd.to_datetime(date.today())

def getYears(start):
    diff = relativedelta(now, start)
    return diff.years

# Add a new column that contains the age of the user
users["Age"] = users['BirthDate'].map(getYears)

TypeError: Cannot compare tz-naive and tz-aware timestamps